In [7]:
from bs4 import BeautifulSoup
import requests

# Khởi tạo một danh sách để lưu trữ các URL
all_urls = []

# Sử dụng vòng lặp để duyệt qua các trang
for i in range(1, 5):
    url = f'https://nhadat24h.net/nha-dat-ban/page{i}'
    
    # Gửi yêu cầu GET đến trang web
    response = requests.get(url)
    
    # Parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Tìm tất cả các thẻ <a> có class là "title" và thuộc tính href
    links = soup.find_all('a', class_='a-title', href=True)
    
    # Kiểm tra xem có ít nhất một thẻ a nào không
    if links:
        # Lặp qua các thẻ a và lấy thuộc tính href
        for link in links:
            href = link['href']
            full_url = 'https://nhadat24h.net' + href
            all_urls.append(full_url)
            
# Ghi các URL vào một file văn bản
file_path = "buy_urls.txt"
with open(file_path, "w", encoding="utf-8") as file:
    for url in all_urls:
        file.write(url + "\n")

print("Đã lưu các URL vào file buy_urls.txt")


Đã lưu các URL vào file buy_urls.txt


In [8]:
#crawl rent link
from bs4 import BeautifulSoup
import requests

# Khởi tạo một danh sách để lưu trữ các URL
all_urls = []

# Sử dụng vòng lặp để duyệt qua các trang
for i in range(1, 5):
    url = f'https://nhadat24h.net/nha-dat-cho-thue/page{i}'
    
    # Gửi yêu cầu GET đến trang web
    response = requests.get(url)
    
    # Parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Tìm tất cả các thẻ <a> có class là "title" và thuộc tính href
    links = soup.find_all('a', class_='a-title', href=True)
    
    # Kiểm tra xem có ít nhất một thẻ a nào không
    if links:
        # Lặp qua các thẻ a và lấy thuộc tính href
        for link in links:
            href = link['href']
            full_url = 'https://nhadat24h.net' + href
            all_urls.append(full_url)
            
# Ghi các URL vào một file văn bản
file_path = "rent_urls.txt"
with open(file_path, "w", encoding="utf-8") as file:
    for url in all_urls:
        file.write(url + "\n")

print("Đã lưu các URL vào file rent_urls.txt")


Đã lưu các URL vào file rent_urls.txt


In [9]:
# crawl data
from bs4 import BeautifulSoup
import requests
import json

def extract_text(soup, selector):
    element = soup.select_one(selector)
    return element.text.strip() if element else None

def extract_image_url(soup):
    image_element = soup.select_one("a.swipebox")
    if image_element:
        base_url = "https://nhadat24h.net/"
        image_url = image_element['href']
        return base_url + image_url
    return None

def extract_description(soup):
    description_element = soup.select_one("div.dv-txt-mt.dv-pt-item")
    if description_element:
        # Extract the text and remove any additional tags
        description = ' '.join(description_element.stripped_strings)
        return description
    return None

def crawl_data_from_url(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url.strip(), headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            title = extract_text(soup, "h1#txtcontenttieudetin")

            # Trích xuất giá và diện tích
            price_area_element = soup.select_one("label#ContentPlaceHolder1_ctl00_lbGiaDienTich")
            if price_area_element:
                price_area_text = price_area_element.get_text(strip=True)
                price_area_parts = price_area_text.split('-')
                if len(price_area_parts) == 2:
                    price = price_area_parts[0].strip().split(':')[1].strip()
                    area = price_area_parts[1].strip().split('M²')[0].strip()
                    
            # Trích xuất tình trạng pháp lý
            legal_status = None
            for p in soup.find_all('p'):
                if 'Tình trạng pháp lý' in p.get_text():
                    strong_tag = p.find('strong')
                    if strong_tag:
                        legal_status = strong_tag.text.strip()
                        break

            property_type = extract_text(soup, "i.fas.fa-building + label")
            address = extract_text(soup, "label#ContentPlaceHolder1_ctl00_lbTinhThanh")
            bedrooms = extract_text(soup, "td.col1:-soup-contains('Phòng Ngủ') + td")
            bathrooms = extract_text(soup, "td.col1:-soup-contains('Phòng WC') + td")
            floors = extract_text(soup, "td.col1:-soup-contains('Số tầng') + td")
            direction = extract_text(soup, "td.col1:-soup-contains('Hướng') + td")
            entrance_width = extract_text(soup, "td.col1:-soup-contains('Đường vào') + td")
            frontage = extract_text(soup, "td.col1:-soup-contains('Mặt tiền') + td")
            
            # Trích xuất link ảnh
            image_url = extract_image_url(soup)
            image_links = image_url if image_url else None

            # Trích xuất mô tả
            description = extract_description(soup)

            # Trích xuất thông tin về tỉnh/thành phố và quận/huyện từ URL
            city_tag = soup.select_one("#loadmenuTinhThanh")
            city = city_tag.text.strip() if city_tag else None

            breadcrumbs = soup.select("div.dv-breadcrumb li")
            district = None
            for breadcrumb in breadcrumbs:
                a_tag = breadcrumb.find("a")
                if a_tag and ("Quận" in a_tag.text or "Huyện" in a_tag.text or "Thị xã" in a_tag.text or "Thành phố" in a_tag.text):
                    district = a_tag.text.strip()
                    
                    # Remove "Quận", "Huyện", "Thị xã", "Thành phố"
                    district = district.replace("Quận", "").replace("Huyện", "").replace("Thị xã", "").replace("Thành phố", "").strip()
                    
                    break

            data = {
                "title": title,
                "price": price,
                "area": area,
                "type":"Mua Bán",
                "address": address,
                "description": description,
                "law": legal_status,
                "type_estate": property_type,
                "province": city,
                "district": district,
                "bedroom": bedrooms,
                "toilet": bathrooms,
                "floor": floors,
                "direction": direction,
                "entrance_width": entrance_width,
                "width": frontage,
                "image_links": image_links                
            }

            # Chuyển các trường có giá trị là chuỗi rỗng thành null
            for key, value in data.items():
                if value == "":
                    data[key] = None

            return data
        else:
            print(f"Lỗi khi truy cập vào URL: {url}")
            return None
    except Exception as e:
        print(f"Lỗi khi trích xuất thông tin từ URL: {url}, Lỗi: {e}")
        return None

data_list = []

with open("buy_urls.txt", "r", encoding="utf-8") as file:
    urls = file.readlines()

for url in urls:
    url = url.strip()
    item = crawl_data_from_url(url)
    if item:
        data_list.append(item)

with open("buy_data.json", "w", encoding="utf-8") as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, indent=4)

print("Dữ liệu đã được lưu vào tệp buy_data.json")


Dữ liệu đã được lưu vào tệp buy_data.json


In [ ]:
# crawl rent data
from bs4 import BeautifulSoup
import requests
import json

def extract_text(soup, selector):
    element = soup.select_one(selector)
    return element.text.strip() if element else None

def extract_image_url(soup):
    image_element = soup.select_one("a.swipebox")
    if image_element:
        base_url = "https://nhadat24h.net/"
        image_url = image_element['href']
        return base_url + image_url
    return None

def extract_description(soup):
    description_element = soup.select_one("div.dv-txt-mt.dv-pt-item")
    if description_element:
        # Extract the text and remove any additional tags
        description = ' '.join(description_element.stripped_strings)
        return description
    return None

def crawl_data_from_url(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url.strip(), headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            title = extract_text(soup, "h1#txtcontenttieudetin")

            # Trích xuất giá và diện tích
            price_area_element = soup.select_one("label#ContentPlaceHolder1_ctl00_lbGiaDienTich")
            if price_area_element:
                price_area_text = price_area_element.get_text(strip=True)
                price_area_parts = price_area_text.split('-')
                if len(price_area_parts) == 2:
                    price = price_area_parts[0].strip().split(':')[1].strip()
                    area = price_area_parts[1].strip().split('M²')[0].strip()
                    
            # Trích xuất tình trạng pháp lý
            legal_status = None
            for p in soup.find_all('p'):
                if 'Tình trạng pháp lý' in p.get_text():
                    strong_tag = p.find('strong')
                    if strong_tag:
                        legal_status = strong_tag.text.strip()
                        break

            property_type = extract_text(soup, "i.fas.fa-building + label")
            address = extract_text(soup, "label#ContentPlaceHolder1_ctl00_lbTinhThanh")
            bedrooms = extract_text(soup, "td.col1:-soup-contains('Phòng Ngủ') + td")
            bathrooms = extract_text(soup, "td.col1:-soup-contains('Phòng WC') + td")
            floors = extract_text(soup, "td.col1:-soup-contains('Số tầng') + td")
            direction = extract_text(soup, "td.col1:-soup-contains('Hướng') + td")
            entrance_width = extract_text(soup, "td.col1:-soup-contains('Đường vào') + td")
            frontage = extract_text(soup, "td.col1:-soup-contains('Mặt tiền') + td")
            
            # Trích xuất link ảnh
            image_url = extract_image_url(soup)
            image_links = image_url if image_url else None

            # Trích xuất mô tả
            description = extract_description(soup)

            # Trích xuất thông tin về tỉnh/thành phố và quận/huyện từ URL
            city_tag = soup.select_one("#loadmenuTinhThanh")
            city = city_tag.text.strip() if city_tag else None

            breadcrumbs = soup.select("div.dv-breadcrumb li")
            district = None
            for breadcrumb in breadcrumbs:
                a_tag = breadcrumb.find("a")
                if a_tag and ("Quận" in a_tag.text or "Huyện" in a_tag.text or "Thị xã" in a_tag.text or "Thành phố" in a_tag.text):
                    district = a_tag.text.strip()
                    
                    # Remove "Quận", "Huyện", "Thị xã", "Thành phố"
                    district = district.replace("Quận", "").replace("Huyện", "").replace("Thị xã", "").replace("Thành phố", "").strip()
                    
                    break

            data = {
                "title": title,
                "price": price,
                "area": area,
                "type":"Cho Thuê",
                "address": address,
                "description": description,
                "law": legal_status,
                "type_estate": property_type,
                "province": city,
                "district": district,
                "bedroom": bedrooms,
                "toilet": bathrooms,
                "floor": floors,
                "direction": direction,
                # "entrance_width": entrance_width,
                "width": frontage,
                "image_links": image_links                
            }

            # Chuyển các trường có giá trị là chuỗi rỗng thành null
            for key, value in data.items():
                if value == "":
                    data[key] = None

            return data
        else:
            print(f"Lỗi khi truy cập vào URL: {url}")
            return None
    except Exception as e:
        print(f"Lỗi khi trích xuất thông tin từ URL: {url}, Lỗi: {e}")
        return None

data_list = []

with open("rent_urls.txt", "r", encoding="utf-8") as file:
    urls = file.readlines()

for url in urls:
    url = url.strip()
    item = crawl_data_from_url(url)
    if item:
        data_list.append(item)

with open("rent_data.json", "w", encoding="utf-8") as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, indent=4)

print("Dữ liệu đã được lưu vào tệp rent_data.json")
